### LLM Graph RAG for Financial Data ###
by 

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from datetime import datetime
import tqdm
import os
import torch
from datasets import load_from_disk, load_dataset
from utils import *
import json
from neo4j import GraphDatabase
import glob
from timeit import default_timer as timer
from time import sleep
from string import Template
import re
import pandas as pd
import getdatafin3

In [3]:
# Load Model
os.environ['CUDA_VISIBLE_DEVICES'] = "5,6"

base_model = AutoModelForCausalLM.from_pretrained(
    'meta-llama/Llama-2-7b-chat-hf',
    trust_remote_code=True,
    #device_map="auto",
    torch_dtype=torch.float32,   # optional if you have enough VRAM
)
base_model.model_parellal = True
model = PeftModel.from_pretrained(base_model, 'FinGPT/fingpt-forecaster_dow30_llama2-7b_lora')
model = model.eval()
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-chat-hf')
tokenizer.padding_side = "right"
tokenizer.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
# Database Config
NEO4J_CONNECTION_URL='neo4j+s://7db9df71.databases.neo4j.io'
NEO4J_USER='neo4j'
NEO4J_PASSWORD='HeQXE-OQgB-g4ECIqsZu9UTyqEaH2XfxZjcHd3PimkU'

neo4j_url = NEO4J_CONNECTION_URL
neo4j_user = NEO4J_USER
neo4j_pass = NEO4J_PASSWORD

gds = GraphDatabase.driver(neo4j_url,auth=(neo4j_user,neo4j_pass))

In [5]:
#Se genera set de datos con los ajustes para generar los prompts para pasar por LLM y generar el data set
com_list = pd.read_csv('./data/ticker_select.txt', sep='\t',names=['symbol', 'cipk'])
data = getdatafin3.get_data('data',com_list)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
  3%|▎         | 4/134 [00:34<14:46,  6.82s/it]

 'EntityCommonStockSharesOutstanding' in symbol GOOG


[*********************100%%**********************]  1 of 1 completed
  4%|▍         | 6/134 [00:44<11:53,  5.57s/it]

 'EntityCommonStockSharesOutstanding' in symbol META


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
  9%|▉         | 12/134 [01:33<13:08,  6.47s/it]

...

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
 98%|█████████▊| 131/134 [18:38<00:20,  6.99s/it]

 FinnhubAPIException(status_code: 403): You don't have access to this resource. in symbol AI.PA


 99%|█████████▊| 132/134 [18:39<00:10,  5.11s/it]

 'EntityCommonStockSharesOutstanding' in symbol ABNB


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
100%|██████████| 134/134 [18:59<00:00,  8.51s/it]


In [79]:
#Generación del prompt base para pasar al LLM y genere nuestro data set base
prompt = """[INST]<<SYS>>
You are a seasoned stock market analyst. Your task is to create strategic questions for companies based on relevant news and basic financials from the past weeks, then provide an answer for the questions for the companies' financial status.
0. ALWAYS FINISH THE OUTPUT. Never send partial responses
1. The output should look like :
{
    "question": "Where are Apple Inc. offices?",
    "answer": "The offices are located in ONE APPLE PARK WAY Cupertino CA"
}

Or like :
{
    "question": "When where the stock value of Apple Inc. the highest?",
    "answer": "Apple Inc. highest value was on 2023-12-14"
}
2. Provide at least 20 and no more than 50 questions with their corresponding answers.

<<SYS>>
$ctext
"""

In [80]:
#Proceso local para generar nuestros data set
def process_local(file_prompt,system_msg):
    inputs = tokenizer(
        file_prompt, return_tensors='pt',
        padding=False, max_length=4096
    )
    inputs = {key: value.to(model.device) for key, value in inputs.items()}
        
    res = model.generate(
        **inputs, max_length=4096, do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
        use_cache=True
    )
    output = tokenizer.decode(res[0], skip_special_tokens=True)
    return output    

#Proceso para chatgpt para generar nuestros data set
def process_chatgpt(file_prompt,system_msg):
    with model.chat_session(system_msg,file_prompt):
        result = model.generate(file_prompt)
    nlp_results = json.dumps(result)
    sleep(8)
    return nlp_results

#generar corpus de preguntas y respuestas
def extract_qa(file, prompt_template,path):
    
    com_list = pd.read_csv(file, sep='\t',names=['symbol', 'cipk'])
    results = []
    start = timer()

    system_msg = "You are a seasoned stock market analyst."
    
    for i in com_list['symbol']:
        
        with open(f'{path}/{i}_data.txt', 'r') as file:
            text = file.read().replace('\n', '')
            print(f"Extracting questions and anwers for {i}")
            prompt = Template(prompt_template).substitute(ctext=json.dumps(text))
            result = process_local(prompt, system_msg=system_msg)
            results.append(result)
            
    end = timer()
    print(f"Pipeline completed in {end-start} seconds")
    return results